In [1]:
import torch
import torch.nn as nn
import os
import numpy as np
import nibabel as nib
from torch.utils.data import Dataset
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

model_file = 'L_BLAHFCN_A2B_SH3000_saved_model_split'
dirs = ['H','L','M','N','O']



In [2]:
class SHDataSet(Dataset):
    def __init__(self,X):
        
        self.X = X
        self.to_tensor = transforms.ToTensor()
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, i):
        
        vec_a = np.reshape(self.X[i,:],(1, 16, 1))
        a = self.to_tensor(vec_a)
    
        return a
    

        
  

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1 * 16, 400)
        self.fc2 = nn.Linear(400,200)
        self.fc3 = nn.Linear(200,50)
        self.fc4 = nn.Linear(50,200)
        self.fc5 = nn.Linear(200,15)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x
        
net = Net()
print(net) 

Net(
  (fc1): Linear(in_features=16, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=200, bias=True)
  (fc5): Linear(in_features=200, out_features=15, bias=True)
)


In [4]:
model = net.load_state_dict(torch.load(model_file))

In [5]:
for i in range(len(dirs)):
    
    apply_to_file = "/Users/kurtschilling/Data/harmonization/CDMRI_Challenge_2018/Testing_Data/%s/prisma/st/norm_dwi_SHfitOrder4_EvenOdd2_SH3000.nii.gz" % (dirs[i])
    save_file = "L_BLAHFCN_s%s_A2B_SH3000.nii.gz" % (dirs[i])
    label_path = "/Users/kurtschilling/Data/harmonization/SLANT/Test_%s_PRISMA_seg_2AB.nii.gz" % (dirs[i])

    print(apply_to_file)
    print(save_file)
    print(label_path)

    # load and reshape
    nifti_path = os.path.normpath(apply_to_file)
    img = nib.load(nifti_path)
    data = img.get_fdata()
    dims = data.shape
    print(dims)
    #X = np.reshape(data,(dims[0]*dims[1]*dims[2],dims[3]))
    #print(X.shape)
    #print(X.dtype)
    #X.astype(float)
    #print(X.dtype)
    
    label_path = os.path.normpath(label_path)
    l_img = nib.load(label_path)
    l_data = l_img.get_fdata()
    print(l_data.shape)
    
    X = np.zeros((dims[0]*dims[1]*dims[2],16))
    index = 0
    for i in range(dims[0]):
        for j in range(dims[1]):
            for k in range(dims[2]):
                temp_sh_ip = data[i,j,k,:]
                temp = np.array(temp_sh_ip)
                temp = temp.T
                
                label_voxel = l_data[i,j,k]
                X[index,0] = label_voxel
                X[index,1:] = temp
                index += 1
                    
                              
    print(X.shape)
    print(X.dtype)
    X.astype(float)
    print(X.dtype)
    
    shset = SHDataSet(X)
    print(len(shset))
    
    Y = np.zeros((dims[0],dims[1],dims[2],dims[3]))
    print(Y.shape)
    Y = np.reshape(Y,(dims[0]*dims[1]*dims[2],dims[3]))
    print(Y.shape)
             
    batch_idx = 0
    for batch_idx, (data) in enumerate(shset):
    
        data = Variable(data).float()
    
        net_out = net(data)
        b = net_out.detach().numpy()
        c = np.reshape(b,(1,15))
        #print(c.shape)
        Y[batch_idx,:]=c;
    
    
        if batch_idx % 10000 == 0:
            print(batch_idx)
            #print(b)
    
    Y = np.reshape(Y,(dims[0],dims[1],dims[2],dims[3]))
    print(Y.shape)
    
    new_img = nib.Nifti1Image(Y,img.affine,img.header)
    nib.save(new_img,save_file)

/Users/kurtschilling/Data/harmonization/CDMRI_Challenge_2018/Testing_Data/H/prisma/st/norm_dwi_SHfitOrder4_EvenOdd2_SH3000.nii.gz
L_BLAHFCN_sH_A2B_SH3000.nii.gz
/Users/kurtschilling/Data/harmonization/SLANT/Test_H_PRISMA_seg_2AB.nii.gz
(96, 96, 60, 15)
(96, 96, 60)
(552960, 16)
float64
float64
552960
(96, 96, 60, 15)
(552960, 15)
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
(96, 96, 60, 15)
/Users/kurtschilling/Data/harmonization/CDMRI_Challenge_2018/Testing_Data/L/prisma/st/norm_dwi_SHfitOrder4_EvenOdd2_SH3000.nii.gz
L_BLAHFCN_sL_A2B_SH3000.nii.gz
/Users/kurtschilling/Data/harmonization/SLANT/Test_L_PRISMA_seg_2AB.nii.gz
(96, 96, 60, 15)
(96, 96, 60)
(552960